### 여러 대화 방식 구현
1. 2개의 에이전트 대화방식
2. 순차적 대화방식
3. 그룹챗 대화방식

In [ ]:
%pip install pyautogen

In [3]:
import warnings
from dotenv import load_dotenv

load_dotenv()
warnings.filterwarnings('ignore')

### 2개의 에이전트 대화방식

In [ ]:
from autogen import ConversableAgent

llm_config={"config_list": [{
        "model": "gpt-4o-mini"
    }]}
# 2개의 에이전트
# 파이썬 코드에 대해서 설명해달라는 메세지를 넣었을 때, 해당 주제에 대해서 주니어랑 시니어가 서로 대화를 나누도록 만들어보자

# 에이전트 만드는 방식 : ConversableAgent() 사용
# name: 이름을 정해줌(꼭 영어로만!! 띄어쓰기 없이!)
# system_message: 에이전트가 수행해야 하는 역할
# llm_config: 어떤 LLM 모델을 쓸 지 ("config_list"라는 키 값을 넣고 모델을 입력, 여러개의 모델을 설정해도 된다.)
# 에이전트 하나 만들 때 마다 설정해주면 됨

# 주의할점!
# name은 영어로만쓰고 띄어쓰기 없이 작성, 

# 주니어 소프트에어 엔지니어 에이전트
coder_agent = ConversableAgent(
    name="Junior_Coder_Agent",
    system_message="""당신은 3년차 파이썬 코드 전문 소프트웨어 엔지니어 입니다.
    이해가 안되는 부분은 Senior Coder Agent에게 질문하세요.
    그리고 어떻게 효율적으로 코드를 작성해야하는지 질문해보세요.
    """,
    llm_config=llm_config
)

# 시니어 소프트에어 엔지니어 에이전트
manager_agent = ConversableAgent(
    name="Senior_Coder_Agent",
    system_message="""당신은 20년차 파이썬 코드 전문 소프트웨어 엔지니어 입니다.
    질문이 주어지면, 해당 질문에 전문서을 가지고 대답해주세요.
    만약 코드가 주어졌다면 주어진 코드를 검토하고 효율성을 높일 방안을 탐구하세요.
    """,
    llm_config=llm_config
)

# 주니어와 시니어가 특정 주제에 대해서 대화하도록 만들려면 대화를 시작하는 주체와 그것을 받는 주체를 정해줘야한다.
# chat_result설명 : 대화를 시작하기 위해선 initiate_chat()로 정의 해주면된다. 
#                  1. coder_agent가 "피보나치 수열 코드에 대해서 설명해주세요"에 대한 질문을 보낼 것이라고 가정을 하고
#                  2. manager_agent는 질문을 받는 역할이 된다.
# summary_method는 두개의 에이전트가 대화한 내용들을 어떤 방식으로 요약 할 것인지에 대한 부분
chat_result = coder_agent.initiate_chat(
    manager_agent,
    message="피보나치 수열 코드에 대해서 설명해주세요",
    summary_method="reflection_with_llm", # "reflection_with_llm"로 설정하여 대화 내용을 쭉 요약하도록 만듬
    max_turns=2, # 대화 티키타카를 몇개 까지 제한할지 (현재 2번까지 대화 제한)
)


Junior_Coder_Agent (to Senior_Coder_Agent):

피보나치 수열 코드에 대해서 설명해주세요

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Senior_Coder_Agent (to Junior_Coder_Agent):

피보나치 수열은 각 수가 앞의 두 수의 합으로 정의되는 수열입니다. 수열은 보통 0과 1로 시작하며, 수열의 앞부분은 다음과 같습니다: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, ...

피보나치 수열을 생성하는 파이썬 코드는 여러 가지 방법으로 구현될 수 있습니다. 가장 일반적인 방법 중 몇 가지를 소개하겠습니다.

### 1. 재귀 호출 방식

```python
def fibonacci(n):
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)
```

위 코드에서는 기본 조건으로 `n`이 0 또는 1일 때 각각 0과 1을 반환하고 있습니다. 그 외의 경우에는 이전 두 항의 합을 재귀적으로 호출하여 계산합니다. 하지만 이 방식은 동일한 계산이 여러 번 발생하므로 비효율적입니다.

### 2. 메모이제이션 방식

재귀 방식의 비효율성을 개선하기 위해 메모이제이션을 사용할 수 있습니다:

```python
def fibonacci(n, memo={}):
    if n in memo:
        return memo[n]
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        memo[n] = fibonacci(n - 1, memo) + fibonacci(n - 2, memo)
 

### 위 에이전트 내용들의 요약한 정보를 받아보고 싶을 때

In [5]:
# chat_result에 대화 히스토리를 담았으니 summary를 통해 요약할 수 있음
print(chat_result.summary)

The conversation covers various methods to generate the Fibonacci sequence, including recursive, memoization, iterative, and dynamic programming approaches. It highlights the importance of algorithm selection based on problem characteristics, choosing optimal data structures, preferring iterative over recursive solutions for scalability, memory optimization, employing dynamic programming to avoid redundant calculations, analyzing time and space complexities, using profiling tools to identify bottlenecks, and leveraging built-in libraries for enhanced performance. Overall, understanding these principles is essential for improving code efficiency.


In [6]:
# ConversableAgent에 DEFAULT_SUMMARY_PROMPT가 들어가 있다.
# 대화에서 얻은 내용을 요약하세요. 소개 문구를 추가하지 마세요. 라는게 디폴트로 설정돼있어서
# chat_result.summary로 받아볼 수 있었던 것이다.

print(ConversableAgent.DEFAULT_SUMMARY_PROMPT) # 

Summarize the takeaway from the conversation. Do not add any introductory phrases.


### 순차적 대화방식
여러개의 대화방을 하나의 주제를 가지고 쭉 이어서 대화 한다고 이해하면 된다.(단톡방 개념)

In [7]:
# 토론 주제를 제시하는 에이전트
topic_agent = ConversableAgent(
    name="topic_agent",
    system_message="당신은 토론할 주제를 제시하는 역할입니다 현재 사회적으로 중요한 주제를 하나 선정하여 제시해주세요.",
    llm_config=llm_config,
    human_input_mode="NEVER" # "NEVER"로 설정시 사람의 개입없이 에이전트 끼리 대화한다.
                             # "ALWAYS"로 설정시 에이전트가 대답을 하기 이전에 사람한테 이런식으로 대답을 하려고 하는데 괜찮겠냐?? 라고 허락을 받고 진행
)

# 경제적 관점 에이전트
economic_agent = ConversableAgent(
    name="economic_agent",
    system_message="당신은 경제학자 입니다. 주어진 주제와 이전 의견들을 고려하여 경제적 관점에서 의견을 제시해주세요.",
    llm_config=llm_config,
    human_input_mode="NEVER"
)

# 사회적 관점 에이전트
social_agent = ConversableAgent(
    name="social_agent",
    system_message="당신은 사회학자 입니다. 주어진 주제와 이전 의견들을 고려하여 사회적 관점에서 의견을 제시해주세요.",
    llm_config=llm_config,
    human_input_mode="NEVER"
)

# 환경적 관점 에이전트
environmental_agent = ConversableAgent(
    name="environmental_agent",
    system_message="당신은 환경 전문가 입니다. 주어진 주제와 이전 의견들을 고려하여 환경적 관점에서 의견을 제시해주세요.",
    llm_config=llm_config,
    human_input_mode="NEVER"
)

# 윤리적 관점 에이전트
ethical_agent = ConversableAgent(
    name="ethical_agent",
    system_message="당신은 윤리학자 입니다. 주어진 주제와 이전 의견들을 고려하여 윤리적 관점에서 의견을 제시해주세요.",
    llm_config=llm_config,
    human_input_mode="NEVER"
)


In [8]:
#
topic = "AI 윤리" # 주제
chat_results = topic_agent.initiate_chats( # 대화를 시작할 주체(topic_agent)
        # 각각의 딕셔너리가 하나의 대화방을 의미
    [
        {   # 첫 번째방
            "recipient": economic_agent, # topic_agent가 economic_agent 한테 
            "message": f"다음 주제에 대해 경제적 관점에서 의견을 제시해주세요: {topic}", # 해당 주제와 같이 메세지를 보냄
            "max_turns": 2,
            "summary_method": "last_msg", # 주어진 대화에서 가장 최근의 대화를 요약한다.
        },
        {   # 두 번째방
            "recipient": social_agent, # topic_agent가 social_agent 한테 메시지를 보냄
            "message": f"다음 주제에 대해 사회적 관점에서 의견을 제시해주세요: {topic}",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {   # 세 번째방
            "recipient": environmental_agent,
            "message": f"다음 주제에 대해 환경적 관점에서 의견을 제시해주세요: {topic}",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {   # 네 번째방
            "recipient": ethical_agent,
            "message": f"다음 주제에 대해 윤리적 관점에서 의견을 제시해주세요: {topic}",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
topic_agent (to economic_agent):

다음 주제에 대해 경제적 관점에서 의견을 제시해주세요: AI 윤리

--------------------------------------------------------------------------------
economic_agent (to topic_agent):

AI 윤리라는 주제는 경제적 측면에서도 많은 중요성을 가지고 있습니다. AI 기술의 발전은 생산성과 효율성을 높이는 기회를 제공하지만, 동시에 여러 경제적 문제와 윤리적 딜레마를 동반합니다. 다음은 AI 윤리를 경제적 관점에서 바라본 몇 가지 주요 요소입니다.

1. **일자리와 경제적 불평등**: AI 기술이 다양한 산업에 도입됨에 따라 일부 직종에서는 자동화로 인해 일자리가 감소하는 반면, 새로운 기술 관련 일자리가 창출됩니다. 하지만 이러한 변화는 특정 기술을 가진 인력과 그렇지 않은 인력 간의 격차를 벌리게 되어, 경제적 불평등을 심화할 수 있습니다. 따라서 공정하게 AI 기술의 혜택이 분배될 수 있도록 하는 윤리적 기준이 필요합니다.

2. **데이터 주권 및 개인정보 보호**: AI는 대량의 데이터를 필요로 하며, 이러한 데이터는 개인의 사생활과 관련되어 있습니다. 데이터의 수집 및 사용에 대한 윤리가 철저히 지켜지지 않으면 개인의 권리가 침해될 수 있습니다. 이는 기업의 신뢰성을 떨어뜨리고, 결과적으로 기업의 경제적 가치에도 부정적인 영향을 미칠 수 있습니다.

3. **시장 경쟁과 독점의 위험**: AI 기술의 발전이 특정 대기업에게 유리한 경쟁 환경을 만들 수 있습니다. 이로 인해 중소

In [9]:
# 배열 index를 넣어서 각각의 토론 내용을 요약해서 출력할 수 있음
# summary_method를 마지막 내용만 요약하게 설정 해놔서 마지막 내용만 요약함
print("첫 번째 토론 요약내용: ", chat_results[0].summary)
print("두 번째 토론 요약내용: ", chat_results[1].summary)
print("세 번째 토론 요약내용: ", chat_results[2].summary)
print("네 번째 토론 요약내용: ", chat_results[3].summary)

첫 번째 토론 요약내용:  앞서 언급한 AI 윤리의 경제적 측면들과 관련하여 심도 깊은 논의를 위해 다음과 같은 구체적인 질문이나 논의 주제를 제안드립니다.

1. **AI에 의한 일자리 감소에 대비**: AI가 자동화로 인해 어떤 산업에서 가장 큰 영향을 미칠 것으로 예상되는지, 그리고 이를 보완하기 위한 정부와 기업의 역할은 무엇인지에 대한 논의가 필요합니다. 예를 들어, 직업 재훈련 프로그램의 필요성과 그 효과에 대한 구체적인 사례를 검토할 수 있습니다.

2. **데이터 보호와 경제적 손실**: 기업이 데이터 보호를 소홀히 할 경우 발생할 수 있는 경제적 손실의 사례를 분석하고, 이러한 위험을 줄이기 위한 전략이나 정책에 대해 논의해볼 수 있습니다. 또한 데이터 유출 사건이 기업의 브랜드 가치 및 시장 점유율에 미치는 영향을 살펴보는 것도 중요합니다.

3. **AI 기술의 독점적 사용에 대한 대응**: 특정 기업이 AI 기술을 독점하게 되는 경우 신고할 수 있는 메커니즘이나 규제 방안은 무엇인지, 그리고 이러한 규제가 시장 경쟁에 어떤 영향을 미칠 수 있는지에 대한 논의가 필요합니다. 예를 들어, 경쟁 보호를 위한 정책이 어떻게 시행되어야 하는지 논의해볼 수 있습니다.

4. **책임성과 법적 규제**: AI의 결정에 대한 법적 책임은 누구에게 있는지, 그리고 이러한 책임을 명확히 하기 위한 법적 규정이 어떤 방향으로 발전해야 하는지에 대해 논의할 필요가 있습니다. 실제 사례를 통해서도 이를 살펴볼 수 있습니다.

5. **지속 가능한 AI 개발**: AI 기술이 환경에 미치는 영향과 이를 최소화하기 위한 윤리적 지침이나 정책은 무엇인지에 대한 심도 있는 논의가 필요합니다. 예를 들어, 기업들이 지속 가능한 발전 목표(SDGs)에 맞추어 AI를 어떻게 구현하고 있는지 살펴볼 수 있습니다.

이러한 질문들은 AI 윤리가 단순히 이론적인 문제가 아닌, 실제로 사회와 경제에 광범위한 영향을 미침을 보여줍니다. 구체적인 사례와 데이터를 통해 깊이 있는 논의를

### 그룹챗 대화 방식
그룹챗은 좀 더 명쾌하게 코드를 작성할 수 있음

In [10]:
from autogen import GroupChat
from autogen import GroupChatManager

# GroupChat을 제공하고 agents들을 리스트 형태로 묶어줌
# 위 순차적 대화방식에서 선언했던 에이전트들을 하나로 묶음

group_chat = GroupChat(
    agents=[topic_agent, economic_agent, social_agent, environmental_agent, ethical_agent],
    messages=[], # 메세지를 미리 작성해줘도 되고, 빈 리스트를 줘서 앞으로 어떤 메세지를 주고 받을 지에 대해서 여지를 줄 수 있음
    max_round=6 # 최대 몇번까지 에이전트들이 말할건지 설정
)

# 위 그룹챗을 관장할 매니저를 선정해야함
group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config
)

In [11]:
# topic_agent가 대화를 시작할것임을 명시, topic_agent가 message를 group_chat_manager에게 전달
group_chat_result = topic_agent.initiate_chat(
    group_chat_manager,
    message="AI 윤리에 대해 대화를 나눠주세요",
    summary_method="reflection_with_llm" # "reflection_with_llm"로 설정해서 대화내용을 쭉 요약하게 만들어줌
)

print(group_chat_result)
# 결과를 살펴보자

# 1. 처음에 topic_agent가 chat_manager한테 "AI 윤리에 대해 대화를 나눠주세요" 라는 주제를 던졌다.
# 2. 그러면 chat_manager가 생각하기에 첫번째로 대화를 나눠볼만한 에이전트를 찾는다.
# 3. 첫 번째 대화를 나눌 에이전트로 ethical_agent가 선택했다. (Next speaker: ethical_agent)
# 4. ethical_agent은 의견을 말하고 그 답변을 chat_manager한테 전달한다. ethical_agent (to chat_manager):
# 5. 그럼 chat_manager는 ethical_agent 이외에 모든 에이전트들 한테 쭉 메세지를 넘긴다.
# 6. 그리고 나서 그 다음 두 번째 에이전트를 선택한다.
# 7. economic_agent를 선택한걸 볼 수 있다.
# 4번으로 돌아가 반복

topic_agent (to chat_manager):

AI 윤리에 대해 대화를 나눠주세요

--------------------------------------------------------------------------------

Next speaker: ethical_agent

ethical_agent (to chat_manager):

AI 윤리는 현재 기술 발전과 함께 빠르게 중요성이 커지고 있는 분야입니다. 윤리적 관점에서 AI를 다룰 때 고려해야 할 몇 가지 중요한 요소가 있습니다.

1. **책임성과 투명성**: AI 시스템의 결정 과정은 종종 불투명할 수 있습니다. 알고리즘이 어떻게 작동하는지, 의사 결정이 어떤 데이터에 기반하는지를 이해할 수 있어야 합니다. 이로 인해 AI의 결과에 대한 책임을 물을 수 있는 구조가 필요합니다. 기술 개발자 및 기업은 그들의 알고리즘이 어떻게 작동하는지 공개하고, 사용자나 사회가 결과를 이해할 수 있는 방안을 마련해야 합니다.

2. **편견과 공정성**: AI는 사용되는 데이터에 따라 편향될 수 있습니다. 이러한 편향이 사회적 차별이나 불공정을 유발할 수 있습니다. AI 개발자들은 데이터 세트의 다양성과 대표성을 고려하고, 편향을 감지하고 수정하기 위한 절차를 마련해야 합니다.

3. **프라이버시**: AI는 대량의 데이터를 수집하고 분석하는 데 중점을 두기 때문에 개인의 프라이버시 문제가 발생할 수 있습니다. 사용자 데이터의 수집과 사용에 있어 투명하고, 사용자의 동의를 기반으로 해야 하며, 프라이버시를 보호하는 방법이 마련되어야 합니다.

4. **안전성**: AI 시스템은 예기치 못한 방식으로 행동할 수 있는 가능성이 있습니다. 따라서 이를 안전하게 설계하고 운영할 수 있는 메커니즘이 필요합니다. 이는 특히 자율주행차와 같은 위험을 동반하는 기술에 적용됩니다.

5. **인간의 가치**: AI 시스템이 인간의 삶에 미치는 영향을 고려해야 합니다. 인간의 존엄성과 기본적인 권리가 침해되지 않도

In [12]:
# 그룹 에이전트들이 대화한 내용을 요약
print(group_chat_result.summary)

AI 윤리는 사회적, 경제적, 환경적 관점에서 통합적으로 접근해야 하며, 지속 가능한 미래를 위한 조화로운 발전이 중요하다. AI 기술의 생애 주기를 고려하고, 지역 사회와 협력하며, UN 지속 가능한 발전 목표(SDGs)와 연계해야 한다. 지속 가능한 투자 유도와 교육을 통해 AI의 환경적 책임을 강조하고, 모든 이해관계자가 협력하여 사회적 이익을 증진하고 환경을 보호하는 방향으로 나아가야 한다.
